## Emoji Prediction on Tweets

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import tensorflow.keras

In [2]:
df = pd.read_csv("data/emoji_uncleaned.csv").sample(n=30)
df.dropna(inplace=True)

X = df["Tweet"].values
y = df["Label"].values

In [3]:
emoji_raw = open('data/us_mapping.txt','r',encoding="utf8")

emojis=[]
for sentence in emoji_raw:
    sentence = sentence.rstrip()
    emojis.append(sentence)

    
emoji_dict={}

for e in emojis:
    idx = int(e.split()[0])
    emoji = e.split()[1]
    emoji_dict[idx] = emoji

In [4]:
from keras.preprocessing.sequence import pad_sequences

tokenizer = pickle.load(open("Model/tweet_tokenizer",'rb'))

def preprocess_text(X):
    max_len=40
    X_seqs = tokenizer.texts_to_sequences(X)
    X_seqs_pd = pad_sequences(X_seqs, truncating="pre", padding="pre", maxlen=max_len)
    return X_seqs_pd

In [5]:
from tensorflow import keras

model = keras.models.load_model("Model/BLSTM.h5", compile=False)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 40, 128)           8022016   
_________________________________________________________________
bidirectional (Bidirectional (None, 40, 160)           133760    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 40, 160)           154240    
_________________________________________________________________
global_max_pooling1d (Global (None, 160)               0         
_________________________________________________________________
dropout (Dropout)            (None, 160)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                10304     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0

In [6]:
X_sequences = preprocess_text(X)
predictions = np.argmax(model.predict(X_sequences), axis=1)

y_map = [emoji_dict[idx] for idx in y]
predictions = [emoji_dict[idx] for idx in predictions]

results = pd.DataFrame({"Tweet":X})
results["True"] = y_map
results["Predicted"] = predictions

results

,Tweet,True,Predicted
0,"November 15, 1994 #albumsiappreciate @ Staten ...",🔥,🔥
1,It's hot af today people out here in #atx ain ...,🔥,🔥
2,we weddings @ The Reserve at Augusta,💜,💜
3,@user @ Joshua Tree National Park,📷,📷
4,My people Thanks for coming to Michigan with m...,💕,💕
5,I'm really Doing this shit.. #MichealJacksonJa...,🔥,🔥
6,lmaooo I didn't even notice Santa Clause behin...,😍,😍
7,Sharing our backyard with this guy...he &amp; ...,😊,💕
8,️ these ppl! .....#kbbq #friends #friendsmas #...,❤️,❤️
9,Rise &amp; Grind #motivation #goals #resilienc...,😘,😘


#    

## Emojify Tweet

In [7]:
import string
import re

from tensorflow import keras
emoji_predict_model = keras.models.load_model("Model/BLSTM.h5", compile=False)


def tweet_clean(tweet):
    tweet = str(tweet).lower()
    rm_mention = re.sub(r'@[A-Za-z0-9]+', '', tweet)                       # remove @mentions
    rm_rt = re.sub(r'RT[/s]+', '', rm_mention)                             # remove RT
    rm_links = re.sub(r'http\S+', '', rm_rt)                               # remove hyperlinks
    rm_links = re.sub(r'https?:\/\/\S+','', rm_links)
    rm_nums = re.sub('[0-9]+', '', rm_links)                               # remove numbers
    rm_punc = [char for char in rm_nums if char not in string.punctuation] # remove punctuations
    rm_punc = ''.join(rm_punc)
    cleaned = rm_punc
    
    return cleaned


def predict_emoji(text, model=emoji_predict_model):
    text = tweet_clean(text)
    X_sequences = preprocess_text([text])
    predictions = np.argmax(model.predict(X_sequences), axis=1)
    emoji_idx = predictions[0]
    emoji = emoji_dict[emoji_idx]
    
    return emoji

In [8]:
text = input("Enter tweet \n\n")

print("\n\n Emojified Tweet \n\n")
print(text+" "+predict_emoji(text))

Enter tweet 

Elon is a cool guy @spacex


 Emojified Tweet 


Elon is a cool guy @spacex 😎


In [8]:
text = input("Enter tweet \n\n")

print("\n\n Emojified Tweet \n\n")
print(text+" "+predict_emoji(text))

Enter tweet 

Had great fun this weekend #christmas


 Emojified Tweet 


Had great fun this weekend #christmas 🎄


In [33]:
text = input("Enter tweet \n\n")

print("\n\n Emojified Tweet \n\n")
print(text+" "+predict_emoji(text))

Enter tweet 

What a scene @LA, everyone should visit this place


 Emojified Tweet 


What a scene @LA, everyone should visit this place 📷


In [41]:
text = input("Enter tweet \n\n")

print("\n\n Emojified Tweet \n\n")
print(text+" "+predict_emoji(text))

Enter tweet 

lets hope the wildfires ceases


 Emojified Tweet 


lets hope the wildfires ceases 🔥


In [14]:
text = input("Enter tweet \n\n")

print("\n\n Emojified Tweet \n\n")
print(text+" "+predict_emoji(text))

Enter tweet 

I could not stop laughing after seeing this funny meme


 Emojified Tweet 


I could not stop laughing after seeing this funny meme 😂
